In [3]:
# EDA
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

pd.set_option('display.float_format', lambda x: '%.2f' % x) # Dica: para visualizar os valores continuos muito altos sem formato cientifico

# ML
from sklearn.cluster import KMeans # Pacocte de Clusterização
from sklearn.metrics import silhouette_score, pairwise_distances # ALgoritmos de Clusterização
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder # Normalizar dados numericos de escalas diferentes, variaveis categorias e variaveis ordinais
from sklearn.compose import ColumnTransformer # Transformar algumas colunas antes da clusterização

import optuna

### Carregar dados

In [4]:
df_clientes = pd.read_csv('.\dataset\clientes_pj.csv')

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Carolina\AppData\Local\Temp\ipykernel_4288\3996757017.py:1: SyntaxWarning: invalid escape sequence '\d'
  df_clientes = pd.read_csv('.\dataset\clientes_pj.csv')


In [5]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 23.6+ KB


In [6]:
df_clientes.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao
0,Comércio,713109.95,12,Rio de Janeiro,6,1
1,Comércio,790714.38,9,São Paulo,15,0
2,Comércio,1197239.33,17,São Paulo,4,9
3,Indústria,449185.78,15,São Paulo,6,0
4,Agronegócio,1006373.16,15,São Paulo,15,8
5,Serviços,1629562.41,16,Rio de Janeiro,11,4
6,Serviços,771179.95,13,Vitória,0,1
7,Serviços,707837.61,16,São Paulo,10,6
8,Comércio,888983.66,17,Belo Horizonte,10,1
9,Indústria,1098512.64,13,Rio de Janeiro,9,3


### EDA

In [26]:
df_clientes.describe()

,faturamento_mensal,numero_de_funcionarios,idade,inovacao,clusters
count,500.00,500.00,500.00,500.00,500.00
mean,1026715.63,13.69,9.25,4.39,0.94
std,420609.46,3.12,2.96,2.90,0.77
min,18421.22,2.00,0.00,0.00,0.00
25%,763253.58,12.00,7.00,2.00,0.00
50%,1022957.08,14.00,9.00,4.00,1.00
75%,1295888.52,16.00,11.00,7.00,2.00
max,2390677.22,21.00,16.00,9.00,2.00


In [7]:
# Distribuição da variavel inovação em percentual
percentual_inovacao = df_clientes.value_counts('inovacao') / len(df_clientes) * 100
px.bar(percentual_inovacao, color=percentual_inovacao.index)

In [8]:
# Teste ANOVA (Análise de Variância)
# Verificar se há variações significativas na média de faturamento mensal para diferentes níveis de inovação
# Suposições / Pressupostos:
# - Observações independentes
# - Variável dependente é continua
# - Segue uma distribuição normal
# - Homogenidade das Variâncias
# - Amostras sejam de tamanhos iguais


In [9]:
# Checar se as variâncias (faturamento) entre os grupos (inovação) são homogênias
# Aplicar Teste de Bartlett
# H0 - Variâncias são iguais
# H1 - Variâncias não são iguais

from scipy.stats import bartlett

# Separando os dados de faturamento em grupos com base na coluna 'inovação'
dados_agrupados = [df_clientes['faturamento_mensal'][df_clientes['inovacao'] == grupo] for grupo in df_clientes['inovacao'].unique()] # Cria-se listas de faturamento com base em cada nivel de inovação

# Executar teste de Bartlett
bartlett_test_statistc, bartlett_p_value = bartlett(*dados_agrupados) # Retorna dois parametros depois realizar a função que recebera a lista quebrada em varias listas como argumento da função

# Exibindo resultados
print(f'Estatistica do Test de Bartlett: {bartlett_test_statistc}')
print(f'P-value do Test de Bartlett: {bartlett_p_value}')

Estatistica do Test de Bartlett: 10.901203117231173
P-value do Test de Bartlett: 0.28254182954905804


- temos que aceitar H0, pois o p-value é maior que 0.05

In [10]:
# Executar o Teste de Shapiro-Wilk
# Verificar se os dados seguem uma distribuição normal
# H0 - Segue uma distribuição normal
# H1 - Não segue uma distribuição normal

from scipy.stats import shapiro

# Executar o teste
shapiro_test_statistc, shapiro_p_value = shapiro(df_clientes['faturamento_mensal'])

# Exibindo Resultados
print(f'Estatistica do Teste SW: {shapiro_test_statistc}')
print(f'P-Value do Teste SW: {shapiro_p_value}')

Estatistica do Teste SW: 0.9959857602472711
P-Value do Teste SW: 0.23513451034389005


- P-Value maior que 0.05 confirma H0

In [11]:
# Aplicar a ANOVA de Welch, pois as amostras são de tamanhos diferentes
# H0 - Não há diferenças significativas entres as médias dos grupos
# H1 - Há pelo menos um diferenças significativas entres as médias dos grupos
from pingouin import welch_anova

aov = welch_anova(dv='faturamento_mensal', between='inovacao', data=df_clientes)

# Exibindo Resultados
print(f'Estatistica do Teste de ANOVA de Welch: {aov.loc[0, 'F']}')
print(f'P-Value do Teste de ANOVA de Welch: {aov.loc[0, 'p-unc']}')

Estatistica do Teste de ANOVA de Welch: 1.1269836194061693
P-Value do Teste de ANOVA de Welch: 0.34526211273911467


- P-value > 0.05 temos que aceitar H0

Nesta etapa, buscamos investigar se o nível de inovação das empresas tem relação estatisticamente significativa com seu faturamento.

- A variável "inovação" apresentou distribuição equilibrada entre os grupos.
- Testes de Bartlett e Shapiro-Wilk confirmaram que as premissas de homogeneidade de variâncias e normalidade são atendidas.
- A ANOVA de Welch, usada como validação adicional para grupos de tamanhos diferentes, indicou que não há diferença significativa no faturamento médio entre os níveis de inovação (p > 0.05).

**Conclusão:** Com base nos dados disponíveis, não encontramos evidências de que o nível de inovação por si só influencie o faturamento das empresas. Isso reforça a importância de buscar novas variáveis explicativas ou explorar segmentações mais refinadas, como faremos na etapa de clusterização.


### Treinar o Algoritmo K-Means

In [12]:
# Selecionar as colunas para clusterização
X = df_clientes.copy()

# Separando variáveis numéricas, categóricas e ordinais
numeric_features = ['faturamento_mensal', 'numero_de_funcionarios', 'idade']
categorical_features = ['localizacao', 'atividade_economica']
ordinal_features = ['inovacao']

# Aplicar Transformaçãoes por tipo
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()
ordinal_transformer = OrdinalEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('ord', ordinal_transformer, ordinal_features)
    ]
)

# Transformar os dados
X_transformed = preprocessor.fit_transform(X)


In [13]:
X_transformed

array([[-0.74634498, -0.54179191, -1.10058849, ...,  0.        ,
         0.        ,  1.        ],
       [-0.56165548, -1.5035527 ,  1.94344851, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.40582654,  1.06114274, -1.77704115, ...,  0.        ,
         0.        ,  9.        ],
       ...,
       [ 2.8196246 , -1.18296577,  0.25231684, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.03321411, -0.54179191, -1.43881482, ...,  0.        ,
         0.        ,  3.        ],
       [-2.03011486, -0.22120498, -1.77704115, ...,  1.        ,
         0.        ,  9.        ]])

- Foi feito a transformação de todas as variáveis para que possamos encontrar a melhor configuração de clusters usando a métrica de Silhouette Score como nossa métrica de validação

In [14]:
# Optuna para Otimização de Hiperparâmetros
# Criando função para treinar o modelo e extrair uma métrica

def kmeans_objective(trail):
    # Definindo os hiperparâmetros a serem ajustados
    n_clusters = trail.suggest_int('n_clusters', 3, 10)
    distance_metrics = trail.suggest_categorical('distance_metrics', ['euclidean', 'minkowski'])

    # Criando Modelo
    modelo_kmeans = KMeans(n_clusters=n_clusters, random_state=51)

    # Treinar o Modelo
    modelo_kmeans.fit(X_transformed)

    # Calculando o Silhouette Score
    distances = pairwise_distances(X_transformed, metric=distance_metrics)
    silhouette_avg = silhouette_score(distances, modelo_kmeans.labels_)

    return silhouette_avg




In [15]:
# Criar um estudo do Optuna
search_space = {'n_clusters': [3, 4, 5, 6, 7, 8, 9, 10], 'distance_metrics': ['euclidean', 'minkowski']}
sampler = optuna.samplers.GridSampler(search_space=search_space)
estudo_kmeans = optuna.create_study(direction='maximize', sampler=sampler)

# Rodar estudo
estudo_kmeans.optimize(kmeans_objective, n_trials=100)

[I 2025-06-05 10:51:21,406] A new study created in memory with name: no-name-98d09014-db4c-4503-8756-a1bc7fabd2c5
[I 2025-06-05 10:51:24,455] Trial 0 finished with value: 0.3847879496502923 and parameters: {'n_clusters': 4, 'distance_metrics': 'euclidean'}. Best is trial 0 with value: 0.3847879496502923.
[I 2025-06-05 10:51:24,475] Trial 1 finished with value: 0.1123004838158908 and parameters: {'n_clusters': 9, 'distance_metrics': 'euclidean'}. Best is trial 0 with value: 0.3847879496502923.
[I 2025-06-05 10:51:24,495] Trial 2 finished with value: 0.44454582909990875 and parameters: {'n_clusters': 3, 'distance_metrics': 'minkowski'}. Best is trial 2 with value: 0.44454582909990875.
[I 2025-06-05 10:51:24,512] Trial 3 finished with value: 0.3847879496502923 and parameters: {'n_clusters': 4, 'distance_metrics': 'minkowski'}. Best is trial 2 with value: 0.44454582909990875.
[I 2025-06-05 10:51:24,537] Trial 4 finished with value: 0.14731572416665714 and parameters: {'n_clusters': 8, 'dis

Após o pré-processamento das variáveis numéricas e categóricas com técnicas adequadas (padronização, codificação nominal e ordinal), estruturamos uma função de treino para o algoritmo KMeans com suporte à experimentação de hiperparâmetros.

Utilizamos o framework Optuna para buscar a melhor combinação de:
- Número de clusters (de 3 a 10)
- Métrica de distância (euclidiana ou Minkowski)

Cada configuração foi avaliada com base no **Silhouette Score**, que mede a coesão e separação dos grupos.

**Vantagens dessa abordagem:**
- Modularidade e reprodutibilidade do pipeline
- Comparação justa entre configurações
- Otimização automatizada para ganho de tempo e qualidade dos resultados


### Analisar Resultados

In [16]:
# Melhor configuração encontrada pelo Optuna
best_params = estudo_kmeans.best_params
# Instanciando o modelo K-Means com os melhores parâmetros
best_kmeans = KMeans(n_clusters=best_params['n_clusters'], random_state=51)
best_kmeans.fit(X_transformed)

# Calculando o Silhouette Score
distances = pairwise_distances(X_transformed, metric=best_params['distance_metrics'])
best_silhouette = silhouette_score(distances, best_kmeans.labels_)

print(f'k (Número de Clusters): {best_params['n_clusters']}')
print(f'Métrica de Distancia Selecionada: {best_params['distance_metrics']}')
print(f'Silhouette Score: {best_silhouette}')

k (Número de Clusters): 3
Métrica de Distancia Selecionada: euclidean
Silhouette Score: 0.4445458290999088


In [17]:
# Criar coluna com cluster escolhido
df_clientes['clusters'] = best_kmeans.labels_

In [18]:
# Visualizar os primeiros registros
df_clientes.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao,clusters
0,Comércio,713109.95,12,Rio de Janeiro,6,1,0
1,Comércio,790714.38,9,São Paulo,15,0,0
2,Comércio,1197239.33,17,São Paulo,4,9,1
3,Indústria,449185.78,15,São Paulo,6,0,0
4,Agronegócio,1006373.16,15,São Paulo,15,8,1
5,Serviços,1629562.41,16,Rio de Janeiro,11,4,2
6,Serviços,771179.95,13,Vitória,0,1,0
7,Serviços,707837.61,16,São Paulo,10,6,1
8,Comércio,888983.66,17,Belo Horizonte,10,1,0
9,Indústria,1098512.64,13,Rio de Janeiro,9,3,2


### Visualizar Resultados

In [27]:
# Cruzar idade e faturamento, apresentando os clusters
px.scatter(df_clientes, x='idade', y='faturamento_mensal', color='clusters')

In [20]:
# Cruzar inovação e faturamento, apresentando clusters
px.scatter(df_clientes, x='inovacao', y='faturamento_mensal', color='clusters')

Após otimizar os hiperparâmetros com Optuna, os melhores resultados foram:

- **Número de clusters (k):** 3
- **Métrica de distância:** Euclidiana
- **Silhouette Score:** 0.4445

Esse resultado indica que os agrupamentos possuem uma separação satisfatória e podem ser analisados como perfis distintos de clientes.

Para entender o comportamento dos clusters, adicionamos a coluna `cluster` ao dataset original e geramos visualizações segmentadas:

#### Interpretação Visual dos Clusters

Analisamos os grupos identificados pelo KMeans a partir de duas variáveis-chave: idade e inovação — sempre cruzadas com o faturamento.

#### Idade vs Faturamento
- **Cluster 0:** perfil jovem com faturamento médio; pode representar consumidores em ascensão.
- **Cluster 1:** clientes mais velhos com menor receita; grupo menos ativo economicamente.
- **Cluster 2:** diversidade etária com alta receita; perfil premium e potencial para personalização de alto valor.

#### Inovação vs Faturamento
- **Cluster 0:** moderadamente inovador, com receita média.
- **Cluster 1:** baixo interesse em inovação, baixo faturamento.
- **Cluster 2:** grupo inovador e economicamente relevante; ideal para estratégias de lançamento e fidelização.

**Conclusão:** Os clusters revelam perfis distintos que podem embasar estratégias personalizadas de produto, marketing e relacionamento, especialmente o cluster 2, com alto potencial de valor e abertura à inovação.



### Salvar o Modelo e o Pipeline de Tranformação

In [21]:
import joblib

# Salvar o Modelo
joblib.dump(best_kmeans, 'modelo_clusterizacao_cliente.pkl')
joblib.dump(preprocessor, 'pipeline_clusterizacao_cliente.pkl')

['pipeline_clusterizacao_cliente.pkl']

### Aplicação Batch no Gradio

In [22]:
import gradio as gr

modelo = joblib.load('.\modelo_clusterizacao_cliente.pkl')
preprocessor = joblib.load('.\pipeline_clusterizacao_cliente.pkl')

# Função aplicação batch
def clustering(arquivo):
    # Carregar o CSV em um DataFrame
    df_empresas = pd.read_csv(arquivo.name)
    # Transformar os dados do DF para o formato que o KMeans precisa
    X_transformed = preprocessor.fit_transform(df_empresas)
    # Treinar Modelo
    modelo.fit(X_transformed)
    # Criar a coluna cluster no DF
    df_empresas['cluster'] = modelo.labels_
    # Salvando como CSV
    df_empresas.to_csv('.\clusters.csv', index=False)
    # return 'arquivo'
    return '.\clusters.csv'


<>:3: SyntaxWarning:

invalid escape sequence '\m'

<>:4: SyntaxWarning:

invalid escape sequence '\p'

<>:17: SyntaxWarning:

invalid escape sequence '\c'

<>:19: SyntaxWarning:

invalid escape sequence '\c'

<>:3: SyntaxWarning:

invalid escape sequence '\m'

<>:4: SyntaxWarning:

invalid escape sequence '\p'

<>:17: SyntaxWarning:

invalid escape sequence '\c'

<>:19: SyntaxWarning:

invalid escape sequence '\c'

C:\Users\Carolina\AppData\Local\Temp\ipykernel_4288\3352261481.py:3: SyntaxWarning:

invalid escape sequence '\m'

C:\Users\Carolina\AppData\Local\Temp\ipykernel_4288\3352261481.py:4: SyntaxWarning:

invalid escape sequence '\p'

C:\Users\Carolina\AppData\Local\Temp\ipykernel_4288\3352261481.py:17: SyntaxWarning:

invalid escape sequence '\c'

C:\Users\Carolina\AppData\Local\Temp\ipykernel_4288\3352261481.py:19: SyntaxWarning:

invalid escape sequence '\c'



#### Interface

In [23]:
app = gr.Interface(
    clustering,
    gr.File(file_types=[".csv"]),
    "file"
)

app.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [24]:
# Criar Interface
with gr.Blocks() as app:
    gr.Markdown("## Upload de Arquivo CSV para Clustering")
    file_input = gr.File(label="Selecione o arquivo CSV", file_types=[".csv"]), 
    output= gr.Textbox(label='Resultado do Clustering')

    def process_file(file):
        return clustering(file)
    
    submit_btn = gr.Button("Processar")
    submit_btn.click(process_file, inputs=file_input, outputs=output)

# Rodar aplicação
app.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [25]:
print(gr.__version__)

5.33.0
